In [1]:
from Cryptodome.Util.number import inverse, GCD
from Cryptodome.Random.random import randint
from asn1 import Encoder, Decoder, Numbers
from pygost import gost34112012256
from os.path import exists

In [2]:
def generateSignFile(E, P, q, r, s, Q, p, A, B):
    Qx, Qy = lift(Q[0]), lift(Q[1])
    file = Encoder()
    file.start()
    file.enter(Numbers.Sequence)
    file.enter(Numbers.Set)
    file.enter(Numbers.Sequence)
    file.write(b'\x80\x06\x07\x00', Numbers.OctetString)
    file.write(b'GOST R 34.10-2018. Variant 8. Signature', Numbers.UTF8String)
    file.enter(Numbers.Sequence)
    file.write(Qx, Numbers.Integer)
    file.write(Qy, Numbers.Integer)
    file.leave()
    file.enter(Numbers.Sequence)
    file.enter(Numbers.Sequence)
    file.write(p, Numbers.Integer)
    file.leave()
    file.enter(Numbers.Sequence)
    file.write(A, Numbers.Integer)
    file.write(B, Numbers.Integer)
    file.leave()
    file.enter(Numbers.Sequence)
    file.write(Px, Numbers.Integer)
    file.write(Py, Numbers.Integer)
    file.leave()
    file.write(q, Numbers.Integer)
    file.leave()
    file.enter(Numbers.Sequence)
    file.write(r, Numbers.Integer)
    file.write(s, Numbers.Integer)
    file.leave()
    file.leave()
    file.enter(Numbers.Sequence)
    file.leave()
    file.leave()
    file.leave()
    with open("signedFile.asn1", "wb") as signedData:
        signedData.write(file.output())

In [3]:
def parseSignFile(signedFile):
    dataToParse = b''
    with open(signedFile, "rb") as plaintText:
        for line in plaintText:
            dataToParse += line
    file = Decoder()
    file.start(dataToParse)
    file.enter()
    file.enter() 
    file.enter()
    file.read()
    file.read()
    file.enter()  
    Qx = file.read()[1]  
    Qy = file.read()[1]  
    file.leave()
    file.enter()
    file.enter()  
    p = file.read()[1]  
    file.leave()
    file.enter() 
    A = file.read()[1]  
    B = file.read()[1]  
    file.leave()
    file.enter() 
    Px = file.read()[1]  
    Py = file.read()[1] 
    file.leave()
    q = file.read()[1]
    file.leave()
    file.enter()
    r = file.read()[1]  
    s = file.read()[1]  
    file.leave()
    file.leave()
    file.enter()
    file.leave()
    file.leave()
    file.leave()
    return p, A, B, Px, Py, Qx, Qy, q, r, s

In [4]:
def generateSignature(fileName, p, A, B, Px, Py, r, d):
    dataToSign = b''
    with open(fileName, "rb") as plainText:
        for line in plainText:
            dataToSign += line
    E = EllipticCurve(GF(p), [A, B])
    P = E([Px, Py])
    Q = d * P
    h = int(gost34112012256.new(dataToSign).hexdigest(), base=16)
    e = h % r
    if e == 0:
        e = 1
    while 1:
        k = randint(0, r)
        C = k * P
        Cx = lift(C[0])
        q = Cx % r
        if q == 0:
            continue
        s = (q * d + k * e) % r
        if s == 0:
            continue
        generateSignFile(E, P, r, q, s, Q, p, A, B)
        return

def checkSignature(signedFile):
    p, A, B, Px, Py, Qx, Qy, r, q, s = parseSignFile(signedFile)
    dataToCheck = b''
    with open("test.txt", "rb") as plainText:
        for line in plainText:
            dataToCheck += line
    h = int(gost34112012256.new(dataToSign).hexdigest(), base=16)
    if q < 0 or q > r or s < 0 or s > r:
        print("Message was changed!")
        return
    e = h % r
    if e == 0:
        e = 1
    v = inverse(e, r)
    z1 = (s * v) % r
    z2 = ((-q) * v) % r
    E = EllipticCurve(GF(p), [A, B])
    P = E([Px, Py])
    Q = E([Qx, Qy])
    C = z1 * P + z2 * Q
    Cx = lift(C[0])
    R = Cx % r
    if q == R:
        print("OK")
    else:
        print("Message was compromised!!")

In [5]:
p =  57896044628958718631213028275518411328476149599789770738757218840632915517411  
r =  28948022314479359315606514137759205664236832023231628035871193493020981068937  
A =  -1
B =  44516423948019661825420813927965592341675839270849860441861020678502941837466
Px =  2323576058601956664720966708045726308916627824741707729836708887517232685058
Py =  20772302011053991390127435262297715010367018383131467831609444907978987653753
d = 22735942951758587535782280423639958017385652232174266686574137212303591377090

In [6]:
while 1:
    flag = int(input("1. Sign file.\n2. Check Sign.\n3. Generate Keys again.\n"))
    if flag == 1:
        if exists("test.txt"):
            generateSignature("test.txt", p, A, B, Px, Py, r, d)
        else:
            print("Create file test.txt!\n")
    elif flag == 2:
        if exists("signedFile.asn1"):
            checkSignature("signedFile.asn1")
        else:
            print("Sign something first!\n")
    elif flag == 3:
        while True:
            d = randint(1,r - 1)
            if GCD(d, r) == 1:
                print("Key was changed!")
            break

1. Sign file.
2. Check Sign.
3. Generate Keys again.
3
Key was changed!
1. Sign file.
2. Check Sign.
3. Generate Keys again.
2
Sign something first!

1. Sign file.
2. Check Sign.
3. Generate Keys again.
1


NameError: name 'EllipticCurve' is not defined